# Mining Media Data I - Assignment 01
## Winter Semester 25/26

**Prof. Dr. Rafet Sifa | Armin Berger | Dr. Lorenz Sparrenberg**

---

### Introduction

In this assignment, we will investigate methods to mine frequent itemsets, implement the Apriori and the rule extraction algorithms, and apply them to analyze patterns in transactional data. All programming assignments are implemented using Python 3.10+.

---

## Part 1: Implementation of the Apriori and the Rule Extractor Algorithms (8 Pts.)

In this part, using Python, we will implement:
1. The **Apriori algorithm** to extract frequent itemsets
2. The **rule extraction algorithm** to extract association rules from frequent itemsets

We will test the algorithms on the dataset from Table 1 by:
- Extracting frequent itemsets for **Smin = 0.1**
- Extracting association rules for **Cmin = 0.3**
- Calculating the **Lift value** for each rule
- Visualizing results using scatter plots with Support, Confidence, and Lift values

### 1.1 Import Required Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import combinations
from collections import defaultdict

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

Matplotlib is building the font cache; this may take a moment.


### 1.2 Transactional Database - Table 1

**Table 1:** A transactional DB T for Part 1

The dataset contains 8 players with their purchased items:
- **Items:** Elixir, Shield, Gem, Sword, Wand, Giant Wand

In [2]:
# Create the transactional database from Table 1
transactions = [
    ['Elixir', 'Shield'],                      # Player 1
    ['Gem', 'Shield'],                         # Player 2
    ['Elixir', 'Sword'],                       # Player 3
    ['Elixir', 'Wand', 'Shield'],             # Player 4
    ['Elixir', 'Sword'],                       # Player 5
    ['Wand', 'Shield'],                        # Player 6
    ['Elixir', 'Wand', 'Shield', 'Sword'],    # Player 7
    ['Elixir', 'Giant Wand']                   # Player 8
]

# Convert to DataFrame for better visualization
df = pd.DataFrame({
    'Player ID': [f'Player {i+1}' for i in range(len(transactions))],
    'Bought Items': [', '.join(t) for t in transactions]
})

print("Transactional Database:")
print(df)
print(f"\nTotal transactions: {len(transactions)}")

Transactional Database:
  Player ID                 Bought Items
0  Player 1               Elixir, Shield
1  Player 2                  Gem, Shield
2  Player 3                Elixir, Sword
3  Player 4         Elixir, Wand, Shield
4  Player 5                Elixir, Sword
5  Player 6                 Wand, Shield
6  Player 7  Elixir, Wand, Shield, Sword
7  Player 8           Elixir, Giant Wand

Total transactions: 8


### 1.3 Implementation of the Apriori Algorithm

The Apriori algorithm finds frequent itemsets using the principle that:
> **All subsets of a frequent itemset must also be frequent.**

The algorithm works in passes:
1. **Pass 1:** Count 1-itemsets and determine which are frequent
2. **Pass k:** Generate candidate k-itemsets from frequent (k-1)-itemsets, then count and prune
3. Continue until no new frequent itemsets are found

In [3]:
def calculate_support(itemset, transactions):
    """Calculate support for an itemset"""
    count = sum(1 for transaction in transactions if set(itemset).issubset(set(transaction)))
    return count / len(transactions)

def get_frequent_1_itemsets(transactions, min_support):
    """Get frequent 1-itemsets"""
    # Count occurrences of each item
    item_counts = defaultdict(int)
    for transaction in transactions:
        for item in transaction:
            item_counts[item] += 1
    
    # Filter by minimum support
    num_transactions = len(transactions)
    frequent_items = {}
    for item, count in item_counts.items():
        support = count / num_transactions
        if support >= min_support:
            frequent_items[frozenset([item])] = support
    
    return frequent_items

def apriori_gen(frequent_itemsets_prev, k):
    """Generate candidate k-itemsets from (k-1)-itemsets"""
    candidates = set()
    itemsets = list(frequent_itemsets_prev.keys())
    
    for i in range(len(itemsets)):
        for j in range(i + 1, len(itemsets)):
            # Join step: merge itemsets that differ by one item
            union = itemsets[i] | itemsets[j]
            if len(union) == k:
                candidates.add(union)
    
    return candidates

def apriori(transactions, min_support):
    """
    Apriori algorithm to find all frequent itemsets
    
    Parameters:
    - transactions: list of transactions (each transaction is a list of items)
    - min_support: minimum support threshold
    
    Returns:
    - Dictionary of frequent itemsets with their support values
    """
    # Get frequent 1-itemsets
    frequent_itemsets = get_frequent_1_itemsets(transactions, min_support)
    all_frequent_itemsets = frequent_itemsets.copy()
    
    k = 2
    while frequent_itemsets:
        # Generate candidates
        candidates = apriori_gen(frequent_itemsets, k)
        
        # Prune candidates and calculate support
        frequent_itemsets = {}
        for candidate in candidates:
            support = calculate_support(candidate, transactions)
            if support >= min_support:
                frequent_itemsets[candidate] = support
        
        # Add to all frequent itemsets
        all_frequent_itemsets.update(frequent_itemsets)
        k += 1
    
    return all_frequent_itemsets

print("Apriori algorithm implemented successfully!")

Apriori algorithm implemented successfully!


### 1.4 Extract Frequent Itemsets with Smin = 0.1

We now apply the Apriori algorithm to extract all frequent itemsets from Table 1 with a minimum support threshold of **Smin = 0.1**.

In [4]:
# Set minimum support
min_support = 0.1

# Run Apriori algorithm
frequent_itemsets = apriori(transactions, min_support)

# Display results organized by itemset size
print(f"Frequent Itemsets (min_support = {min_support}):")
print("=" * 60)

# Group by itemset size
itemsets_by_size = defaultdict(list)
for itemset, support in frequent_itemsets.items():
    itemsets_by_size[len(itemset)].append((itemset, support))

# Display sorted by size and support
for size in sorted(itemsets_by_size.keys()):
    print(f"\n{size}-Itemsets:")
    print("-" * 60)
    # Sort by support (descending)
    sorted_itemsets = sorted(itemsets_by_size[size], key=lambda x: x[1], reverse=True)
    for itemset, support in sorted_itemsets:
        items = ', '.join(sorted(itemset))
        print(f"  {{{items}}} : support = {support:.3f}")

print(f"\nTotal frequent itemsets found: {len(frequent_itemsets)}")

Frequent Itemsets (min_support = 0.1):

1-Itemsets:
------------------------------------------------------------
  {Elixir} : support = 0.750
  {Shield} : support = 0.625
  {Sword} : support = 0.375
  {Wand} : support = 0.375
  {Gem} : support = 0.125
  {Giant Wand} : support = 0.125

2-Itemsets:
------------------------------------------------------------
  {Elixir, Sword} : support = 0.375
  {Shield, Wand} : support = 0.375
  {Elixir, Shield} : support = 0.375
  {Elixir, Wand} : support = 0.250
  {Shield, Sword} : support = 0.125
  {Elixir, Giant Wand} : support = 0.125
  {Sword, Wand} : support = 0.125
  {Gem, Shield} : support = 0.125

3-Itemsets:
------------------------------------------------------------
  {Elixir, Shield, Wand} : support = 0.250
  {Elixir, Shield, Sword} : support = 0.125
  {Shield, Sword, Wand} : support = 0.125
  {Elixir, Sword, Wand} : support = 0.125

4-Itemsets:
------------------------------------------------------------
  {Elixir, Shield, Sword, Wand} : 

### 1.5 Implementation of the Rule Extraction Algorithm

From the frequent itemsets, we extract association rules and calculate:
- **Support:** Frequency of the itemset (A ∪ B)
- **Confidence:** Conditional probability P(B|A) = Support(A ∪ B) / Support(A)
- **Lift:** Lift(A → B) = Confidence(A → B) / Support(B)

A lift value > 1 indicates a positive correlation between items.

In [5]:
def generate_association_rules(frequent_itemsets, min_confidence):
    """
    Generate association rules from frequent itemsets
    
    Parameters:
    - frequent_itemsets: dictionary of frequent itemsets with their support
    - min_confidence: minimum confidence threshold
    
    Returns:
    - List of rules with their metrics (support, confidence, lift)
    """
    rules = []
    
    # Only consider itemsets with 2 or more items
    for itemset, support_itemset in frequent_itemsets.items():
        if len(itemset) < 2:
            continue
        
        # Generate all possible rules from this itemset
        for i in range(1, len(itemset)):
            # Generate all combinations of size i as antecedent
            for antecedent in combinations(itemset, i):
                antecedent = frozenset(antecedent)
                consequent = itemset - antecedent
                
                # Calculate confidence: support(A ∪ B) / support(A)
                support_antecedent = frequent_itemsets.get(antecedent, 0)
                if support_antecedent == 0:
                    continue
                
                confidence = support_itemset / support_antecedent
                
                # Check minimum confidence
                if confidence >= min_confidence:
                    # Calculate lift: confidence / support(B)
                    support_consequent = frequent_itemsets.get(consequent, 0)
                    if support_consequent > 0:
                        lift = confidence / support_consequent
                    else:
                        lift = 0
                    
                    rules.append({
                        'antecedent': antecedent,
                        'consequent': consequent,
                        'support': support_itemset,
                        'confidence': confidence,
                        'lift': lift
                    })
    
    return rules

print("Association rule extraction algorithm implemented successfully!")

Association rule extraction algorithm implemented successfully!


### 1.6 Extract Association Rules with Cmin = 0.3

We now extract association rules from the frequent itemsets with a minimum confidence threshold of **Cmin = 0.3**.

For each rule, we calculate and display:
- **Support**
- **Confidence**
- **Lift**

In [6]:
# Set minimum confidence
min_confidence = 0.3

# Generate association rules
rules = generate_association_rules(frequent_itemsets, min_confidence)

# Display rules
print(f"Association Rules (min_confidence = {min_confidence}):")
print("=" * 80)

# Sort rules by confidence (descending)
rules_sorted = sorted(rules, key=lambda x: (x['confidence'], x['lift']), reverse=True)

for i, rule in enumerate(rules_sorted, 1):
    antecedent_str = ', '.join(sorted(rule['antecedent']))
    consequent_str = ', '.join(sorted(rule['consequent']))
    
    print(f"\nRule {i}: {{{antecedent_str}}} => {{{consequent_str}}}")
    print(f"  Support:    {rule['support']:.3f}")
    print(f"  Confidence: {rule['confidence']:.3f}")
    print(f"  Lift:       {rule['lift']:.3f}")

print(f"\n{'=' * 80}")
print(f"Total rules found: {len(rules)}")

Association Rules (min_confidence = 0.3):

Rule 1: {Shield, Sword} => {Elixir, Wand}
  Support:    0.125
  Confidence: 1.000
  Lift:       4.000

Rule 2: {Shield, Sword} => {Wand}
  Support:    0.125
  Confidence: 1.000
  Lift:       2.667

Rule 3: {Sword, Wand} => {Elixir, Shield}
  Support:    0.125
  Confidence: 1.000
  Lift:       2.667

Rule 4: {Elixir, Shield, Sword} => {Wand}
  Support:    0.125
  Confidence: 1.000
  Lift:       2.667

Rule 5: {Wand} => {Shield}
  Support:    0.375
  Confidence: 1.000
  Lift:       1.600

Rule 6: {Gem} => {Shield}
  Support:    0.125
  Confidence: 1.000
  Lift:       1.600

Rule 7: {Elixir, Wand} => {Shield}
  Support:    0.250
  Confidence: 1.000
  Lift:       1.600

Rule 8: {Sword, Wand} => {Shield}
  Support:    0.125
  Confidence: 1.000
  Lift:       1.600

Rule 9: {Elixir, Sword, Wand} => {Shield}
  Support:    0.125
  Confidence: 1.000
  Lift:       1.600

Rule 10: {Sword} => {Elixir}
  Support:    0.375
  Confidence: 1.000
  Lift:       1

### 1.7 Visualization - Scatter Plot

**Visualization requirements:**
- X-axis: **Support** values
- Y-axis: **Confidence** values
- **Lift values** incorporated through:
  - **Color intensity:** Higher lift = brighter color
  - **Marker size:** Larger markers indicate higher lift

This allows us to visualize all three metrics (Support, Confidence, and Lift) in a single plot.

In [ ]:
# Extract metrics for plotting
supports = [rule['support'] for rule in rules]
confidences = [rule['confidence'] for rule in rules]
lifts = [rule['lift'] for rule in rules]

# Create scatter plot
fig, ax = plt.subplots(figsize=(12, 8))

# Use lift for both color and size
scatter = ax.scatter(supports, confidences, 
                     c=lifts, 
                     s=[l * 100 for l in lifts],  # Size proportional to lift
                     cmap='viridis', 
                     alpha=0.7,
                     edgecolors='black',
                     linewidth=1.5)

# Add colorbar for lift
cbar = plt.colorbar(scatter, ax=ax)
cbar.set_label('Lift', rotation=270, labelpad=20, fontsize=12)

# Labels and title
ax.set_xlabel('Support', fontsize=12, fontweight='bold')
ax.set_ylabel('Confidence', fontsize=12, fontweight='bold')
ax.set_title('Association Rules: Support vs Confidence (colored and sized by Lift)', 
             fontsize=14, fontweight='bold', pad=20)

# Add grid
ax.grid(True, alpha=0.3, linestyle='--')

# Add reference lines
ax.axhline(y=min_confidence, color='r', linestyle='--', alpha=0.5, label=f'Min Confidence = {min_confidence}')
ax.axvline(x=min_support, color='b', linestyle='--', alpha=0.5, label=f'Min Support = {min_support}')

# Annotate some key rules
for i, rule in enumerate(rules_sorted[:5]):  # Top 5 rules
    antecedent_str = ', '.join(sorted(rule['antecedent']))
    consequent_str = ', '.join(sorted(rule['consequent']))
    label = f"{{{antecedent_str}}} => {{{consequent_str}}}"
    
    ax.annotate(label, 
                xy=(rule['support'], rule['confidence']),
                xytext=(10, 10), 
                textcoords='offset points',
                fontsize=8,
                bbox=dict(boxstyle='round,pad=0.3', facecolor='yellow', alpha=0.5),
                arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0', color='black'))

ax.legend(loc='lower right', fontsize=10)
plt.tight_layout()
plt.show()

print("\n📊 Visualization Tips:")
print("  - Larger/brighter points indicate higher Lift values")
print("  - Points in the upper-right corner have both high support and confidence")
print("  - Lift > 1 indicates positive correlation between items")

### 1.8 Summary Statistics and Key Insights

Analysis of the results from the Apriori algorithm and rule extraction on Table 1 dataset.

In [ ]:
# Create summary statistics
print("=" * 80)
print("SUMMARY STATISTICS")
print("=" * 80)

print(f"\n📊 Dataset Information:")
print(f"  Total transactions: {len(transactions)}")
print(f"  Unique items: {len(set(item for trans in transactions for item in trans))}")
print(f"  Average items per transaction: {np.mean([len(t) for t in transactions]):.2f}")

print(f"\n🔍 Frequent Itemsets:")
print(f"  Total frequent itemsets: {len(frequent_itemsets)}")
for size in sorted(itemsets_by_size.keys()):
    print(f"  {size}-itemsets: {len(itemsets_by_size[size])}")

print(f"\n📏 Association Rules:")
print(f"  Total rules: {len(rules)}")
print(f"  Average support: {np.mean(supports):.3f}")
print(f"  Average confidence: {np.mean(confidences):.3f}")
print(f"  Average lift: {np.mean(lifts):.3f}")

print(f"\n⭐ Top 5 Rules by Confidence:")
for i, rule in enumerate(rules_sorted[:5], 1):
    antecedent_str = ', '.join(sorted(rule['antecedent']))
    consequent_str = ', '.join(sorted(rule['consequent']))
    print(f"  {i}. {{{antecedent_str}}} => {{{consequent_str}}}")
    print(f"     Conf: {rule['confidence']:.3f}, Lift: {rule['lift']:.3f}")

print("\n💡 Key Insights:")
print("  - Elixir is the most frequently purchased item")
print("  - Shield appears in many itemsets, suggesting it's popular")
print("  - High lift values indicate strong item associations")
print("  - Rules with lift > 1 show items bought together more often than expected")
print("=" * 80)